In [ ]:
# Simple linear regression

In [ ]:
#Import the relevant libraries

In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix




In [ ]:
#Load the data

In [6]:
data = pd.read_csv('data.csv')

In [7]:
data

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [ ]:
# Removing unnecesary features that may alter the accuracy 

In [19]:
data=data.drop(data.iloc[:,11:30],axis=1)
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]



In [ ]:
# Print first 5

In [20]:
data.head()



,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883


In [21]:
def empty(x):
    return sum(x.isnull())
print ("Empty values")
print(data.apply(empty,axis=0))

Empty values
diagnosis                 0
radius_mean               0
texture_mean              0
perimeter_mean            0
area_mean                 0
smoothness_mean           0
compactness_mean          0
concavity_mean            0
concave points_mean       0
symmetry_mean             0
fractal_dimension_mean    0
dtype: int64


In [ ]:
# Check out data type

In [22]:
print(data.dtypes)

diagnosis                  object
radius_mean               float64
texture_mean              float64
perimeter_mean            float64
area_mean                 float64
smoothness_mean           float64
compactness_mean          float64
concavity_mean            float64
concave points_mean       float64
symmetry_mean             float64
fractal_dimension_mean    float64
dtype: object


In [ ]:
# X stores all features except "diagnosis" 
# y stores only "diagnosis feature"
# B value of diagnosis feature is now 0 and M is 1

In [24]:
x=data.iloc[:,1:]
y=data['diagnosis'].copy()
y=y.map({'B':0,'M':1})
y


0      1
1      1
2      1
3      1
4      1
      ..
564    1
565    1
566    1
567    1
568    0
Name: diagnosis, Length: 569, dtype: int64

In [ ]:
#The data is split 30% for testing and 70% training 

In [25]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=12)

In [26]:
print(xtrain.var())


radius_mean                   12.235069
texture_mean                  20.377333
perimeter_mean               578.782970
area_mean                 123307.382973
smoothness_mean                0.000191
compactness_mean               0.002477
concavity_mean                 0.006056
concave points_mean            0.001494
symmetry_mean                  0.000759
fractal_dimension_mean         0.000043
dtype: float64


In [ ]:
# Scalng is required here since it requires calculating gradiente descent and minimum distance or global minimum point 
# Every feature entry is substracted from the mean and the result is divided by the standard deviation
# Normalization is not aplicable here since it only maps values from 0 to 1 and that is it.
    
    

In [61]:
scale=StandardScaler(copy=True,with_mean=True,with_std=True)
scale.fit(xtrain)

xtrain_std=scale.transform(xtrain)
xtest_std=scale.transform(xtest)

#print(xtrain_std.var())

In [ ]:
#PCA is used to create clusters 

In [49]:
pca=PCA(n_components=5,svd_solver='randomized')
pca.fit(xtrain_std)

PCA(n_components=5, svd_solver='randomized')

In [ ]:
#Clustering data starts here

In [54]:
pca_xtrain_std=pca.transform(xtrain_std)
pca_xtest_std=pca.transform(xtest_std)



In [ ]:
#Since data now is clustered,  SVC is the best prediction model to set results into classes
#Linear regression is not needed

In [53]:
svc=SVC(C=0.7, gamma=0.063, kernel='rbf')
svc=svc.fit(pca_xtrain_std,ytrain)
svc.score(pca_xtrain_std,ytrain)

0.949748743718593